In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import glob
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
from pathlib import Path
import re
import requests
from io import StringIO
import random
import pdb

In [2]:
# Upload trips data
trips = pd.read_csv("../input_data/trips.csv",index_col=0)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Otra vivienda,Centro estudios,Otros,Autobús urbano,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,8944.175
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Centro estudios,Compras doméstico-familiares,Estudio,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,8944.175
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Compras doméstico-familiares,Otra vivienda,Compras,Andando,...,1,2 adultos con niño(s),2,2,4,2,2,2,2021,10539.365
3,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,25397.860
4,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27611.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21899,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Gestiones personales,Residencia Habitual,Gestiones Personales,Coche conductor,...,1,Hogar de una persona,1,6,3,1,2,2,2011,21474.460
21900,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,28070.860
21901,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,2,2,2011,25703.710
21902,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Compras personales,Compras,Andando,...,1,2 adultos,2,5,6,2,2,2,2011,13577.460


In [3]:
# Just commute
trips = trips[(trips['Act_Ori'].isin(['Trabajo habitual', 'Asuntos trabajo'])) | (trips['Act_Des'].isin(['Trabajo habitual', 'Asuntos trabajo']))]
trips = trips.reset_index(drop=True)

# Actual population
trips = trips[trips['Año'] == 2021]
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,1,2021,25397.860
1,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27611.510
2,20,20,Donostialdea,Donostialdea,Urnieta,Urnieta,Trabajo habitual,"Ocio, cultura, deporte",Trabajo,Coche conductor,...,1,2 adultos,1,5,1,1,1,1,2021,26229.910
3,20,20,Donostialdea,Donostialdea,Urnieta,Urnieta,"Ocio, cultura, deporte",Trabajo habitual,Ocio,Coche conductor,...,1,2 adultos,1,5,1,1,1,1,2021,28597.060
4,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27765.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,20,20,Debagoiena,Goierri,Bergara,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares con niños,2,5,1,1,2,2,2021,23335.260
2206,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Trabajo habitual,Residencia Habitual,Trabajo,Andando,...,1,Otros hogares con niños,1,3,1,1,2,2,2021,21452.435
2207,20,20,Goierri,Goierri,Zumarraga,Zumarraga,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,Otros hogares con niños,1,3,1,1,2,2,2021,18521.825
2208,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Donostia/San Sebastian,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,1,Hogar de una persona,1,5,1,1,2,2,2021,24602.820


In [4]:
trips['elev'] = trips['elev']/trips['elev'].min()
trips['elev'] = trips['elev'].apply(lambda x: int(x) if isinstance(x, float) else x)

In [5]:
# Expand trips
nuevas_filas = []
for index, row in trips.iterrows():
    valores_columnas = row[trips.columns]
    repeticiones = row['elev']
    nuevas_filas.extend([valores_columnas] * repeticiones)

trips = pd.DataFrame(nuevas_filas, columns=trips.columns)

trips = trips.drop(columns=["elev"])

trips = trips.sample(frac=1).reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Urola Kosta,Usurbil,Orio,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares sin niños,1,4,1,1,2,2,2021,27114.810
1,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Donostia/San Sebastian,"Acompañamiento de ayuda, cuidado",Trabajo habitual,Gestiones Personales,Coche conductor,...,1,2 adultos con niño(s),2,4,1,1,2,2,2021,23028.010
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Trabajo habitual,Residencia Habitual,Trabajo,Bicicleta,...,2,2 adultos,2,5,1,1,1,2,2021,22001.500
3,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Errenteria,Trabajo habitual,Residencia Habitual,Trabajo,EUSKOTREN,...,1,1 adulto con niño(s),2,5,1,1,2,2,2021,21388.620
4,20,20,Debabarrena,Debabarrena,Elgoibar,Elgoibar,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,27839.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,20,20,Donostialdea,Donostialdea,Oiartzun,Oiartzun,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Otros hogares con niños,2,3,1,1,2,2,2021,19801.095
10712,20,20,Donostialdea,Bidasoa,Donostia/San Sebastian,Irun,Trabajo habitual,Residencia Habitual,Trabajo,EUSKOTREN,...,2,2 adultos con niño(s),2,4,1,1,2,2,2021,22888.810
10713,20,20,Debabarrena,Debabarrena,Mutriku,Mutriku,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,2 adultos con niño(s),2,5,1,1,2,2,2021,23644.260
10714,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,1,2 adultos,2,5,1,2,2,2,2021,23336.920


In [6]:
trips = trips[trips['Mun_Ori'] != trips['Mun_Des']]
# trips = trips[(trips['Com_Ori'] == 'Donostialdea') | (trips['Com_Des'] == 'Donostialdea')]
trips = trips[trips['Com_Des'] == 'Donostialdea']
trips = trips[trips['Income'] > 1] # Delete students that make work trips
trips = trips.reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Motos,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income
0,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Donostia/San Sebastian,"Acompañamiento de ayuda, cuidado",Trabajo habitual,Gestiones Personales,Coche conductor,...,1,2 adultos con niño(s),2,4,1,1,2,2,2021,23028.010
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Errenteria,Trabajo habitual,Residencia Habitual,Trabajo,EUSKOTREN,...,1,1 adulto con niño(s),2,5,1,1,2,2,2021,21388.620
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,4,1,1,2,2,2021,24011.460
3,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Urnieta,Trabajo habitual,Residencia Habitual,Trabajo,Taxi,...,1,Otros hogares sin niños,1,5,1,2,2,2,2021,28048.410
4,20,20,Urola Kosta,Donostialdea,Zarautz,Andoain,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,1,Hogar de una persona,1,5,1,1,2,2,2021,24629.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,20,20,Donostialdea,Donostialdea,Andoain,Hernani,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,2,2 adultos,1,6,1,1,2,2,2021,25689.660
2502,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Errenteria,Gestiones personales,Trabajo habitual,Gestiones Personales,Coche conductor,...,2,Otros hogares con niños,1,3,1,1,2,2,2021,21297.335
2503,20,20,Urola Kosta,Donostialdea,Orio,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,1,2 adultos con niño(s),1,5,1,1,2,2,2021,25548.660
2504,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2,Otros hogares sin niños,1,6,1,1,1,2,2021,27651.460


In [7]:
# Add income percentile to each trip taker. This is done to assign a building based on wealthy neighborhoods.

# trips['Percentil'] = trips.groupby('Mun_Ori')['Income'].transform(
#     lambda x: pd.qcut(x, q=[0, 0.25, 0.5, 0.75, 1], labels=[1, 2, 3, 4])
# )
trips['Income_Percentile'] = trips.groupby('Mun_Ori')['Income'].transform(
    lambda x: pd.qcut(x, q=4, labels=False, duplicates='drop') + 1  # labels=False to get numeric labels (1-4)
)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Tipo_familia,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income,Income_Percentile
0,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Donostia/San Sebastian,"Acompañamiento de ayuda, cuidado",Trabajo habitual,Gestiones Personales,Coche conductor,...,2 adultos con niño(s),2,4,1,1,2,2,2021,23028.010,1.0
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Errenteria,Trabajo habitual,Residencia Habitual,Trabajo,EUSKOTREN,...,1 adulto con niño(s),2,5,1,1,2,2,2021,21388.620,1.0
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,2 adultos con niño(s),1,4,1,1,2,2,2021,24011.460,3.0
3,20,20,Donostialdea,Donostialdea,Lasarte-Oria,Urnieta,Trabajo habitual,Residencia Habitual,Trabajo,Taxi,...,Otros hogares sin niños,1,5,1,2,2,2,2021,28048.410,4.0
4,20,20,Urola Kosta,Donostialdea,Zarautz,Andoain,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,Hogar de una persona,1,5,1,1,2,2,2021,24629.120,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,20,20,Donostialdea,Donostialdea,Andoain,Hernani,Residencia Habitual,Asuntos trabajo,Trabajo,Coche conductor,...,2 adultos,1,6,1,1,2,2,2021,25689.660,3.0
2502,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Errenteria,Gestiones personales,Trabajo habitual,Gestiones Personales,Coche conductor,...,Otros hogares con niños,1,3,1,1,2,2,2021,21297.335,1.0
2503,20,20,Urola Kosta,Donostialdea,Orio,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,2 adultos con niño(s),1,5,1,1,2,2,2021,25548.660,2.0
2504,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,Otros hogares sin niños,1,6,1,1,1,2,2021,27651.460,4.0


# Assign LAT and LONG Origin and destination

In [8]:
# Bring buildings

gdf = gpd.read_file("../input_data/buildings_with_section/buildings_w_section.shp")
crs_objetivo = 'EPSG:4326'  # WGS84
gdf = gdf.to_crs(crs_objetivo)

gdf['Centroide'] = gdf['geometry'].centroid

gdf_viviendas = gdf[(gdf['Vivienda'] != 0) | (gdf['Duplex'] != 0)]
gdf_oficinas = gdf[(gdf['Oficina'] != 0)]

C:\Users\iazka\AppData\Local\Temp\ipykernel_5500\1232285268.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Centroide'] = gdf['geometry'].centroid


In [13]:
# Define the probability calculation function
def calcular_probabilidad(similitud):
    return 1 / ((similitud + 1) ** 2)

# trips['Income_Percentile'] = pd.to_numeric(trips['Income_Percentile'], errors='coerce')
# gdf_viviendas['Percentil'] = pd.to_numeric(gdf_viviendas['Percentil'], errors='coerce')

def auxiliar(row, salario_persona):
    probabilidades = []
    clasificacion_edificio = pd.to_numeric(row['Percentil'], errors='coerce')
    similitud = abs(salario_persona - clasificacion_edificio)
    probabilidad = calcular_probabilidad(similitud)
    probabilidades.append(probabilidad)
    return probabilidades

# Create a function to assign buildings efficiently
asignaciones = []
def assign_buildings(row):
    
    gdf_viviendas_1 = gdf_viviendas[gdf_viviendas['Town'] == row['Mun_Ori']].reset_index(drop=True)
    salario_persona = pd.to_numeric(row['Income_Percentile'], errors='coerce')
    probabilidades = auxiliar(gdf_viviendas_1, salario_persona)
    edificio_asignado = random.choices(gdf_viviendas_1['Centroide'], weights=probabilidades)[0]
    asignaciones.append(edificio_asignado)

# Apply the function to each row in trips DataFrame
trips['Edif_Ori'] = trips.apply(assign_buildings, axis=1)

ValueError: The number of weights does not match the population

In [18]:
# Assign ORIGIN, 140 min

# Calculate assigment probability
def calcular_probabilidad(similitud):
    return 1 / ((similitud + 1) ** 2)  # Smaller the similtud, higher the probability

# Assign a building to each person based on probability
asignaciones = []
for _, persona in trips.iterrows():
    salario_persona = pd.to_numeric(persona['Income_Percentile'], errors='coerce')
    gdf_viviendas_1 = gdf_viviendas[gdf_viviendas['Town'] == persona['Mun_Ori']].reset_index(drop=True)
    probabilidades = []
    for _, edificio in gdf_viviendas_1.iterrows():
        clasificacion_edificio = pd.to_numeric(edificio['Percentil'], errors='coerce')
        similitud = abs(salario_persona - clasificacion_edificio)
        probabilidad = calcular_probabilidad(similitud)
        probabilidades.append(probabilidad)
    edificio_asignado = random.choices(gdf_viviendas_1['Centroide'], weights=probabilidades)[0]
    asignaciones.append(edificio_asignado)

trips['Edif_Ori'] = asignaciones

In [19]:
# From WKT to two columns: LONG and LAT

from shapely import wkt
from shapely.geometry import Point

def extraer_longitud(punto):
    return punto.x

def extraer_latitud(punto):
    return punto.y

# from shapely import wkt
# from shapely.geometry import Point

# def extraer_longitud(punto_wkt):
#     punto = wkt.loads(punto_wkt)
#     return punto.x

# def extraer_latitud(punto_wkt):
#     punto = wkt.loads(punto_wkt)
#     return punto.y

trips['O_long'] = trips['Edif_Ori'].apply(extraer_longitud)
trips['O_lat'] = trips['Edif_Ori'].apply(extraer_latitud)


# Eliminar columnas innecesarias

trips = trips.drop(columns=["Edif_Ori"])
trips = trips.reset_index(drop=True)
trips

,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,Modo,...,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,Año,Income,Income_Percentile,O_long,O_lat
0,20,20,Bidasoa,Bidasoa,Irun,Irun,Trabajo habitual,"Ocio, cultura, deporte",Trabajo,Andando,...,4,1,2,2,2,2021,24296.96,3,-1.822133,43.329024
1,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Coche conductor,...,4,1,1,1,2,2021,23983.52,2,-1.965820,43.323980
2,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Trabajo habitual,Compras doméstico-familiares,Trabajo,Andando,...,6,1,1,2,2,2021,24325.91,3,-1.993325,43.304479
3,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Trabajo habitual,Residencia Habitual,Trabajo,Bicicleta,...,5,1,2,2,2,2021,24051.47,2,-1.964278,43.320330
4,20,20,Donostialdea,Donostialdea,Hernani,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,Andando,...,5,1,1,2,2,2021,28031.01,4,-1.975652,43.269075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,4,1,2,2,2,2021,24591.56,3,-1.975557,43.321632
10712,20,20,Debagoiena,Debagoiena,Bergara,Bergara,Residencia Habitual,Trabajo habitual,Trabajo,Andando,...,6,1,2,2,2,2021,24403.67,2,-2.482543,43.114248
10713,20,20,Donostialdea,Urola Kosta,Pasaia,Zarautz,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,4,1,1,2,2,2021,20409.40,1,-1.936559,43.322968
10714,20,20,Donostialdea,Donostialdea,Oiartzun,Pasaia,Trabajo habitual,Residencia Habitual,Trabajo,Coche conductor,...,5,1,1,2,2,2021,25393.26,3,-1.855406,43.296052


In [20]:
trips['D_long'] = -2.033693
trips['D_lat'] = 43.256347

In [21]:
trips.columns

Index(['Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des', 'Mun_Ori', 'Mun_Des',
       'Act_Ori', 'Act_Des', 'Proposito', 'Modo', 'Hora_Ini', 'Dur_Tot',
       'Recur', 'Bill', 'Park', 'Peaje', 'Provincia', 'Municipio', 'Comarca',
       'Per_hog', 'Turismos', 'Motos', 'Tipo_familia', 'Sexo', 'Edad',
       'Actividad', 'crnt_tur', 'crnt_mot', 'crnt_otr', 'Año', 'Income',
       'Income_Percentile', 'O_long', 'O_lat', 'D_long', 'D_lat'],
      dtype='object')

In [22]:
# Delete not interesting columns

eliminar = ['Prov_Ori', 'Prov_Des',
       'Act_Ori', 'Act_Des', 'Proposito', 'Dur_Tot',
       'Bill', 'Park', 'Peaje', 'Provincia','Comarca',
       'crnt_mot', 'crnt_otr']
trips = trips.drop(columns=eliminar)
trips

,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Modo,Hora_Ini,Recur,Municipio,Per_hog,Turismos,...,Edad,Actividad,crnt_tur,Año,Income,Income_Percentile,O_long,O_lat,D_long,D_lat
0,Bidasoa,Bidasoa,Irun,Irun,Andando,19:00,21,Irun,2,2,...,4,1,2,2021,24296.96,3,-1.822133,43.329024,-2.033693,43.256347
1,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Coche conductor,07:45,22,Donostia/San Sebastian,1,2,...,4,1,1,2021,23983.52,2,-1.965820,43.323980,-2.033693,43.256347
2,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Andando,13:00,22,Donostia/San Sebastian,1,1,...,6,1,1,2021,24325.91,3,-1.993325,43.304479,-2.033693,43.256347
3,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Bicicleta,11:40,21,Donostia/San Sebastian,2,1,...,5,1,2,2021,24051.47,2,-1.964278,43.320330,-2.033693,43.256347
4,Donostialdea,Donostialdea,Hernani,Hernani,Andando,15:00,22,Hernani,2,3,...,5,1,1,2021,28031.01,4,-1.975652,43.269075,-2.033693,43.256347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,Donostialdea,Donostialdea,Donostia/San Sebastian,Donostia/San Sebastian,Andando,07:40,22,Donostia/San Sebastian,2,2,...,4,1,2,2021,24591.56,3,-1.975557,43.321632,-2.033693,43.256347
10712,Debagoiena,Debagoiena,Bergara,Bergara,Andando,09:30,23,Bergara,1,1,...,6,1,2,2021,24403.67,2,-2.482543,43.114248,-2.033693,43.256347
10713,Donostialdea,Urola Kosta,Pasaia,Zarautz,Coche conductor,14:30,22,Zarautz,2,3,...,4,1,1,2021,20409.40,1,-1.936559,43.322968,-2.033693,43.256347
10714,Donostialdea,Donostialdea,Oiartzun,Pasaia,Coche conductor,13:00,23,Pasaia,2,2,...,5,1,1,2021,25393.26,3,-1.855406,43.296052,-2.033693,43.256347


In [24]:
trips.to_csv("../input_data/workers_eskuzaitzeta_2k.csv")